In [1]:
import argparse
import glob
from sys import exit, path
from shutil import rmtree
from os.path import join, expanduser, exists
from os import mkdir
import re

import numpy as np
import pandas as pd
import scipy.interpolate as spi
import scipy.signal as sps
import scipy.signal as sps

from bokeh.io import output_file, export_png, export_svgs, show, output_notebook
from bokeh.transform import linear_cmap
from bokeh.plotting import figure
from bokeh.models import ColorBar, ColumnDataSource, Span
from bokeh.layouts import gridplot
from bokeh.palettes import brewer
import colorcet as cc

import bokeh_catplot

path.insert(1, expanduser('~/src/cmocean-bokeh'))
path.insert(1, expanduser('~/src/noexiit/software/analyses'))
from fourier_transform import fft, bokeh_freq_domain
from cmocean_cmaps import get_all_cmocean_colours

output_notebook()

Loading BokehJS ...

# Compute timestamps

I need to use the actual timestamp values, rather than do a conversion from an average framerate. One issue is that the closed-loop and open-loop units for timestamp appear to be a little different. The issue might have to do with the computer that is doing the conversion. My closed-loop experiments' timestamps are captured on the acquisition computer, because I'm doing real-time FicTrac processing, whereas my open-loop expeirments' timestamps are derived offline on my analyses computer. So I guess the real difference between my timestamps is whether FicTrac analyzed them _**in real time**_ or if they were analyzed _**offline**_. 

In [22]:
import time

from analyze_fictrac import parse_dats
from analyze_fictrac import unconcat_df

pson_open_loop = parse_dats("/mnt/2TB/data_in/HK_20200317/pson_open_loop/", 1, 5)
pson_closed_loop = parse_dats("/mnt/2TB/data_in/HK_20200317/pson_closed_loop_yaw_gain_unity/", 1, 5)

The ball_radius argument must be in mm. Confirm by inputting 'y'. Otherwise, hit any other key to quit.y
The ball_radius argument must be in mm. Confirm by inputting 'y'. Otherwise, hit any other key to quit.y


## Offline timestamps

In [31]:
ts_open = unconcat_df(pson_open_loop)[0]["timestamp"]
ts_open.tail()

24023    240056.150272
24024    240066.142203
24025    240076.134135
24026    240086.126066
24027    240096.117997
Name: timestamp, dtype: float64

Conversion to elapsed time in the open-loop case is easy. Just change the milliseconds to seconds:

In [30]:
(ts_open/1000).tail()

24023    240.056150
24024    240.066142
24025    240.076134
24026    240.086126
24027    240.096118
Name: timestamp, dtype: float64

How to get absolute time for open-loop? Let's try this:

In [110]:
import datetime
# t_sys from .log file:
datetime.datetime.fromtimestamp(1587656147396.773926/1000).strftime('%Y_%m_%d %H:%M:%S.%f') 

'2020_04_23 08:35:47.396774'

Nope, that gives me the time at which I ran the FicTrac analyses ... that's not what I want. Maybe try working with the `alt_timestamp` column?

In [115]:
ts_open_alt = unconcat_df(pson_open_loop)[0]["alt_timestamp"]
ts_open_alt.tail()

24023    3.094738e+07
24024    3.094739e+07
24025    3.094739e+07
24026    3.094739e+07
24027    3.094740e+07
Name: alt_timestamp, dtype: float64

In [118]:
td = datetime.timedelta(milliseconds=30947396.774000/1000)
date_object = datetime.datetime.now() + td  
date_object

datetime.datetime(2020, 5, 2, 21, 49, 22, 206059)

## Real-time timestamps

In [32]:
ts_closed = unconcat_df(pson_closed_loop)[0]["timestamp"]
ts_closed.tail()

47070    3.893876e+12
47071    3.893886e+12
47072    3.893906e+12
47073    3.893916e+12
47074    3.893925e+12
Name: timestamp, dtype: float64

In [105]:
# elapsed time:
ts_closed = list(ts_closed)
# know that ts_closed[0] is not a value of 0:
(ts_closed[-1] - ts_closed[0]) / 1e9

475.570271784

In [78]:
# t_sys from .log file:
import datetime
datetime.datetime.fromtimestamp(1584510830149.504150/1000).strftime('%Y-%m-%d %H:%M:%S.%f')

'2020-03-17 22:53:50.149504'

Last value in `frame_cntr` matches last `frame` value in `.dat` file. Means we can take from the `.dat` file the output, which looks like so: 

```
478.976180 process [INF] Frame 47074
478.978774 doSearch [INF] optimum sphere rotation:	0.051 -0.034 -0.014  (err=3.837e+03/its=27)
478.978778 doSearch [DBG] Current sphere orientation:	-0.890 -1.970 -0.498
478.978906 updateSphere [DBG] Sphere ROI match overlap: 100.0%
478.979237 process [INF] Timing grab/opt/map/plot/log/disp: 5.8 / 2.6 / 0.1 / 0.0 / 0.0 / 0.3 ms
478.979240 process [INF] Average frame rate [in/out]: 97.6 [0.0 / 112.6] fps
478.985082 grab [DBG] Frame captured 720x5401 @ 3893925482488.000000 (t_sys: 1584511305664.834961 ms, t_day: 82905664.835000 ms)
478.987053 process [DBG] Processed frame added to input queue (l = 1).
```

The last frame, however, results in an error when trying to grab the timestamp: 

```
478.987085 process [INF] Frame 47075
478.989377 doSearch [INF] optimum sphere rotation:	0.057 -0.027 -0.017  (err=4.463e+03/its=24)
478.989380 doSearch [DBG] Current sphere orientation:	-0.868 -1.964 -0.576
478.989484 updateSphere [DBG] Sphere ROI match overlap: 100.0%
478.989687 process [INF] Timing grab/opt/map/plot/log/disp: 7.8 / 2.3 / 0.1 / 0.0 / 0.0 / 0.2 ms
478.989690 process [INF] Average frame rate [in/out]: 97.1 [0.0 / 95.7] fps
479.987576 grab [ERR] Error grabbing PGR frame! Error was: Spinnaker: Failed waiting for EventData on NEW_BUFFER_DATA event. [-1011]
```

The `@ 3893925482488.000000` example value might be a timestamp from the actual FicTrac camera .... I won't be able to convert this time to anything meaningful. Instead, I'm going to want to extract the `t_sys:` value and use those for alignment. I'm going to want to handle the `"ERROR GRABBING PGR FRAME!"` message, perhaps by outputting a `pandas` `NaN` value. I think I can do that: 

In [ ]:
import time

# TODO:
def get_caltime_from_logs(log):
    """
    Compute the average framerate in Hz from a FicTrac .log file. 
    
    Parameters:
    -----------
    log (str): Path to the FicTrac .log file. 

    Returns:
    --------
    Calendar time from the computer system
    """

    with open (log, "r") as f:

        log_lines = f.readlines()

        hz_lines = []
        for line in log_lines:
            if "frame rate" in line:
                Pull out substring between [in/out] and [:
                result = re.search("\[in/out]: (.*) \[", line)
                hz_lines.append(float(result.group(1)))

    return np.mean(hz_lines)